In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
prefix = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-2/2017_09_22_"
data_dir = prefix + "11_43_45"

In [37]:
blocksPerFile = 6
samplesPerRamp = 6000
rampsPerFile = 1500
samplesPerFile = samplesPerRamp * rampsPerFile

firstFile = 24 + 6 * 10
numberOfFiles = 11
lastFile = firstFile + numberOfFiles * blocksPerFile
nbFilesToCheck = numberOfFiles
nbFilesToRebuild = numberOfFiles

**Note**: with file 84 to 144 one can see a nice migration

### Read the data

In [21]:
def readFile( filename, timeSerie_A, timeSerie_B ):
    fd = open(filename,'rb')
    
    dum = np.fromfile(fd, dtype = np.int16)

    timeSerie_A[:] = dum[ 0 : 2 * samplesPerFile : 2 ]
    timeSerie_B[:] = dum[ 1 : 2 * samplesPerFile : 2 ]
    
    fd.close()

In [22]:
timeSerie_A = np.zeros( samplesPerFile )
timeSerie_B = np.zeros( samplesPerFile )

### Check pulse duration in B channel data

In [6]:
threshold = 6000

In [7]:
for k in range(nbFilesToCheck):
    nb = str(int(k * blocksPerFile + firstFile))
    readFile( data_dir + "/record" + nb +".bin", timeSerie_A, timeSerie_B )
    pulseDuration = np.where(timeSerie_B > threshold)[0].shape[0]
    print("file " + str(nb) + ", pulseDuration =  " + str(pulseDuration))

file 24, pulseDuration =  1070
file 30, pulseDuration =  1095
file 36, pulseDuration =  1114
file 42, pulseDuration =  1084
file 48, pulseDuration =  1088
file 54, pulseDuration =  1082
file 60, pulseDuration =  1039
file 66, pulseDuration =  1000
file 72, pulseDuration =  1000
file 78, pulseDuration =  1000
file 84, pulseDuration =  1000


### Check the triggers in the B channel data

In [8]:
for k in range(nbFilesToCheck):
    nb = str(int(k * blocksPerFile + firstFile))
    readFile( data_dir + "/record" + nb +".bin", timeSerie_A, timeSerie_B )
    triggers = np.where( np.diff(timeSerie_B) > threshold )[0]
    print("file " + str(nb) + ", threshold = " + str(triggers))

file 24, threshold = [8999501]
file 30, threshold = [8999406]
file 36, threshold = [8999292]
file 42, threshold = [8999208]
file 48, threshold = [8999120]
file 54, threshold = [8999038]
file 60, threshold = [8998993]
file 66, threshold = [8998894]
file 72, threshold = [8998839]
file 78, threshold = [8998754]
file 84, threshold = [8998690]


### Rebuild the data files

In [31]:
timeSerie_C = np.zeros( samplesPerFile )
timeSerie_D = np.zeros( samplesPerFile )

In [32]:
rebuilt_A = np.zeros((nbFilesToRebuild, samplesPerFile))
rebuilt_B = np.zeros((nbFilesToRebuild, samplesPerFile))
for k in range(nbFilesToRebuild):
    nb        = str( int( k     * blocksPerFile + firstFile) )
    nb_plus_1 = str( int( (k+1) * blocksPerFile + firstFile) )
    readFile( data_dir + "/record" + nb        +".bin", timeSerie_A, timeSerie_B )
    readFile( data_dir + "/record" + nb_plus_1 +".bin", timeSerie_C, timeSerie_D )
    trigs = np.where( np.diff(timeSerie_B) > threshold )[0]
    trig = trigs[0] + 1

    rebuilt_B[ k, 0 : samplesPerFile - trig ] = timeSerie_B[ trig : ]
    rebuilt_B[ k, samplesPerFile - trig : ] = timeSerie_D[ 0 : trig ]
    
    rebuilt_A[ k, 0 : samplesPerFile - trig ] = timeSerie_A[ trig : ]
    rebuilt_A[ k, samplesPerFile - trig : ] = timeSerie_C[ 0 : trig ]
    
    print("file " + str(nb) + ", trigs = " + str(trigs))

file 84, trigs = [8998690]
file 90, trigs = [8998606]
file 96, trigs = [8998533]
file 102, trigs = [8998465]
file 108, trigs = [8998383]
file 114, trigs = [8998290]
file 120, trigs = [8998216]
file 126, trigs = [8998130]
file 132, trigs = [8998059]
file 138, trigs = [8997974]
file 144, trigs = [8997929]


In [24]:
for k in range(nbFilesToRebuild):
    plt.plot(rebuilt_A[k,0:6000])

In [33]:
A_reshaped = rebuilt_A.reshape(nbFilesToRebuild * rampsPerFile, samplesPerRamp)
B_reshaped = rebuilt_B.reshape(nbFilesToRebuild * rampsPerFile, samplesPerRamp)

In [34]:
coupling_A = np.average(A_reshaped, 0)
A_reshaped -= coupling_A

In [27]:
for k in range(nbFilesToRebuild):
    plt.plot(A_reshaped[k,0:6000])

In [35]:
ifft_a = np.fft.ifft( (A_reshaped[::,0:3000]), axis = 1 )

In [41]:
plt.matshow( 20 * np.log10( np.abs( ifft_a[:,100:500] ) ), aspect='auto' )
plt.title("Track 1 / Flight over 1\nFiles {} to {}".format(firstFile, lastFile) )

In [39]:
np.save(data_dir + '/samplesA_track1_flightover1.npy', A_reshaped)